# Example 3: Housing Resilience of North-East San Francisco

Example 3 shows how **pyrecodes** can integrate with the `SimCenter's R2DTool <https://github.com/NHERI-SimCenter/R2DTool>`_ and assess housing resilience of a community. 

The example is based on `R2DTools's Example 1 <https://nheri-simcenter.github.io/R2D-Documentation/common/user_manual/examples/desktop/E1BasicHAZUS/README.html>`_. R2D Tool outputs initial post-earthquake damage states of all buildings in the considered region following a hypothetical earthquake. **pyrecodes** takes this information and simulates regional recovery considering impeding factors and resource constraints. Further details can be found `here <https://doi.org/10.1016/j.rcns.2022.03.001>`_.

![North-East San Francisco Case Study](./Example%203/Example_3_NorthEastSF.png)

## Input data

Three files are needed to construct the model:
- **Component library** file: input file defining the blueprints for system’s components.
- **System Configuration** file: input file defining the spatial and temporal discretization of the system, component locations, considered resources and resilience calculators.
- **main** file: input file defining the classes and path to files of the component library and the system.

In this Example, these files are in a JSON format.

In [ ]:
from pyrecodes import main
system = main.run('./Example 3/NorthEast_SF_Housing_Main.json')
# system = main.load_system('./Example_3.pickle')
system.calculate_resilience()

In [ ]:
from pyrecodes import Plotter
system.save_as_pickle('Example_3.pickle')
plotter_object = Plotter.Plotter()
x_axis_label = 'Time step [day]'
resource_names = ['Shelter', 'FirstResponderEngineer', 'RepairCrew', 'Money']
for resource_name in resource_names:
    y_axis_label = f'{resource_name} Demand/Supply/Consumption'
    axis_object = plotter_object.setup_lor_plot_fig(x_axis_label, y_axis_label)
    plotter_object.plot_single_resource(list(range(system.START_TIME_STEP, system.time_step+1)), system.resilience_calculators[0].system_supply[resource_name], 
                                        system.resilience_calculators[0].system_demand[resource_name], 
                                        system.resilience_calculators[0].system_consumption[resource_name], axis_object)

axis_object = plotter_object.setup_gantt_chart_fig(x_axis_label, system.components)
plotter_object.plot_gantt_chart(system.components[:20], axis_object)

In [ ]:
from pyrecodes import GeoVisualisator as gvis
import matplotlib.pyplot as plt
from importlib import reload
reload(gvis)
geo_visualisator = gvis.R2D_GeoVisualisator(system.components)
time_steps_to_plot = list(range(1, 50)) + list(range(51, 1500, 4))
for time_step in range(1, 2):
    # geo_visualisator.create_current_state_shapefile(time_step=time_step)    
    geo_visualisator.create_current_state_buildings_and_supply_demand_figure(time_step, system, save=True, dpi=100)
# geo_visualisator.create_recovery_gif(time_steps_to_plot)